In [1]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer, InputExample, losses, models, datasets, evaluation
from torch.utils.data import DataLoader

from sklearn.manifold import TSNE
from matplotlib import pyplot as plt

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np

import torch
import random
import torch
from google.colab import drive
drive.mount("/content/gdrive")

def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 92.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.3 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=c14c1e7d2a36897fcef17c9e3f5e685b5d1d9f1779b9a01cdfd34f124ba39b59
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
Mounted at /content/gdrive


In [2]:
# train_df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

# Load the test dataset into a pandas dataframe.
import h5py
import os
"""
eval_df = pd.read_csv('https://raw.githubusercontent.com/sanromra/imdb_data/main/test_IMDB.tsv', delimiter='\t', header=None)

text_col=eval_df.columns.values[0] 
category_col=eval_df.columns.values[1]
"""
test_data =  h5py.File(os.path.join("gdrive/MyDrive/DL_Experiments/TextExperiments/data/new_data/dbpedia", 'processed_test.h5'), 'r')

x_eval = torch.tensor(np.array(test_data['text_embeddings']))
y_eval = torch.tensor(np.array(test_data['labels']))

In [ ]:
#@title SetFit
# st_model = 'paraphrase-mpnet-base-v2' #@param ['paraphrase-mpnet-base-v2', 'all-mpnet-base-v1', 'all-mpnet-base-v2', 'stsb-mpnet-base-v2', 'all-MiniLM-L12-v2', 'paraphrase-albert-small-v2', 'all-roberta-large-v1']
# num_training = 32 #@param ["8", "16", "32", "54", "128", "256", "512"] {type:"raw"}

# set_seed(0)

# orig_model = SentenceTransformer(st_model)

# X_eval_noFT = orig_model.encode(x_eval, show_progress_bar = True, device = 'mps')

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

#IID

In [10]:
x_train_total = []
x_train_federated = []
y_train_total = []
y_train_federated = []

train_data = h5py.File(os.path.join("gdrive/MyDrive/DL_Experiments/TextExperiments/data/new_data/dbpedia", 'processed_train.h5'), 'r')

x_train_total = torch.tensor(np.array(train_data['text_embeddings']))
y_train_total = torch.tensor(np.array(train_data['labels']))

# net = nn.Sequential(nn.Linear(768, 100),nn.ReLU(),nn.Linear(100, 2)).to('mps')

for i in range(14):
    exp_results = torch.load('gdrive/MyDrive/DL_Experiments/TextExperiments/Federated_Learning/result/dbpedia_noniidv2_silos/dbpedia_noniidv2_silo'+str(i)+ '.pt', map_location = 'cuda')
    x_train_federated.append(exp_results['data'][0][0])
    y_train_federated.append(exp_results['data'][0][1])

In [11]:
x_train_federated = torch.cat(tuple(x_train_federated), dim = 0)
y_train_federated = torch.cat(tuple(y_train_federated), dim = 0)

#Non IID

In [ ]:
import h5py
import os

train_data =  h5py.File(os.path.join("gdrive/MyDrive/DL_Experiments/TextExperiments/data/new_data/yahoo", 'processed_train.h5'), 'r')

x_train_total = torch.tensor(np.array(train_data['text_embeddings']))
y_train_total = torch.tensor(np.array(train_data['labels']))

In [ ]:
exp_results = torch.load('gdrive/MyDrive/DL_Experiments/TextExperiments/Federated_Learning/result/yahoo/whole/res_DM_yahoo_MLP_10ipc.pt', map_location = 'cpu')

In [ ]:
num_classes = 10
ipc = 10 ## Equal to num silos
x_train_split = [[] for i in range(ipc)]
y_train_split = [[] for i in range(ipc)]
for i in range(num_classes):
    x_train_class = x_train_total[y_train_total == i]
    y_train_class = y_train_total[y_train_total == i]
    centers = exp_results['data'][0][0][exp_results['data'][0][1] == i]
    distances = torch.cdist(x_train_class, centers)
    closest_centers = torch.argmin(distances, dim=1)
    for i in range(ipc):
        x_train_split[i].append(x_train_class[closest_centers == i])
        y_train_split[i].append(y_train_class[closest_centers == i])

In [ ]:
x_train_federated = [torch.cat(x_train_split[i]) for i in range(ipc)]
y_train_federated = [torch.cat(y_train_split[i]) for i in range(ipc)]
x_train_federated = torch.cat(tuple(x_train_federated), dim = 0)
y_train_federated = torch.cat(tuple(y_train_federated), dim = 0)

#Train

In [12]:
import torch.nn as nn
embed_dim = 768
hidden_dim = 512
num_classes = 14
net = nn.Sequential(nn.Linear(embed_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            #nn.Linear(hidden_dim, hidden_dim),
            #nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, num_classes))

In [13]:
#X_train_noFT = orig_model.encode(x_train_total, show_progress_bar = True, device = 'cuda')
X_eval_noFT = x_eval
X_train_noFT = x_train_total

In [14]:
from torch.utils.data import TensorDataset, DataLoader

batch_size = 256
# trainset = TensorDataset(torch.Tensor(exp_results1['data'][0][0]),torch.Tensor(exp_results1['data'][0][1])) # create your datset
federated_trainset = TensorDataset(torch.Tensor(x_train_federated),torch.Tensor(y_train_federated)) # create your datset
federated_trainloader = torch.utils.data.DataLoader(federated_trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)

trainset = TensorDataset(torch.Tensor(X_train_noFT),torch.Tensor(y_train_total)) # create your datset
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)

valset = TensorDataset(torch.Tensor(X_eval_noFT),torch.Tensor(y_eval)) # create your datset
valloader = torch.utils.data.DataLoader(valset, batch_size=16, num_workers=0)

In [15]:
X_eval_noFT

tensor([[ 0.1414, -0.2522,  0.0378,  ...,  0.0334, -0.0008, -0.0979],
        [-0.0394, -0.0940,  0.0297,  ..., -0.0087,  0.1567, -0.0236],
        [ 0.1579, -0.1827, -0.0556,  ..., -0.1233, -0.0069, -0.0621],
        ...,
        [-0.1128, -0.0594,  0.0581,  ...,  0.0494, -0.0097, -0.1067],
        [-0.0240, -0.0873,  0.1267,  ..., -0.0947,  0.1797,  0.1440],
        [ 0.1409,  0.1106,  0.0533,  ..., -0.0623, -0.1377, -0.0983]])

In [16]:
sum(y_eval), len(y_eval), len(X_eval_noFT)

(tensor(455000), 70000, 70000)

In [17]:
Epoch = 900
lr_schedule = []#[Epoch//2+1]
lr = 0.01
optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9)#, weight_decay=0.0005)

In [18]:
device = 'cuda'
criterion = nn.CrossEntropyLoss().to(device)

In [19]:
def epoch(mode, dataloader, net, optimizer, criterion):
    loss_avg, acc_avg, num_exp = 0, 0, 0
    net = net.to(device)
    #print("Net: {}".format(net.is_cuda))
    criterion = criterion.to(device)

    if mode == 'train':
        net.train()
    else:
        net.eval()

    for i_batch, datum in enumerate(dataloader):
        img = datum[0].float().to(device)
#         print("Sample: {}".format(img.is_cuda))
        lab = datum[1].long().to(device)
#         print("Label: {}".format(lab.is_cuda))
        n_b = lab.shape[0]

        output = net(img).to(device)
        loss = criterion(output, lab)
        acc = np.sum(np.equal(np.argmax(output.cpu().data.numpy(), axis=-1), lab.cpu().data.numpy()))

        loss_avg += loss.item()*n_b
        acc_avg += acc
        num_exp += n_b

        if mode == 'train':
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    loss_avg /= num_exp
    acc_avg /= num_exp

    return loss_avg, acc_avg

### Training whole dataset 

In [ ]:
for ep in range(Epoch+1):
        loss_train, acc_train = epoch('train', trainloader, net, optimizer, criterion)
        if ep in lr_schedule:
            lr *= 0.1
            optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=0.0005)
        loss, acc = epoch('test', valloader, net, optimizer, criterion)
        print(acc)

0.7024
0.70505
0.7069166666666666
0.7074
0.7086333333333333
0.7094666666666667
0.7096666666666667
0.7103333333333334
0.71155
0.7105833333333333
0.7114833333333334
0.7121666666666666
0.7118666666666666
0.7127666666666667
0.7124333333333334
0.7127666666666667
0.7125666666666667
0.71245
0.71385
0.7131166666666666
0.7128666666666666
0.7135833333333333
0.71315
0.7137833333333333
0.7137833333333333
0.71445
0.7133833333333334
0.7144666666666667
0.7138666666666666
0.7125333333333334
0.7142833333333334
0.7144333333333334
0.7142333333333334
0.7149333333333333
0.7139
0.7144333333333334
0.7137333333333333
0.7143333333333334
0.7139
0.7142666666666667
0.71475
0.7146666666666667
0.7149166666666666
0.7144
0.7146666666666667
0.7136666666666667
0.7137666666666667
0.7147
0.7139166666666666
0.71455
0.7148333333333333
0.7146666666666667
0.7149666666666666
0.7147
0.7140833333333333
0.7142333333333334
0.7144333333333334
0.71455
0.7148833333333333
0.7140333333333333
0.7138833333333333
0.7153
0.71395
0.7155666

In [ ]:
epoch('test', valloader, net, optimizer, criterion)

(0.8586611078302065, 0.7216166666666667)

### Training 10 Silos 

In [24]:
net = nn.Sequential(nn.Linear(embed_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            #nn.Linear(hidden_dim, hidden_dim),
            #nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, num_classes))
lr = 0.01
optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9)#, weight_decay=0.0005)

In [25]:
for ep in range(Epoch+1):
        loss_train, acc_train = epoch('train', federated_trainloader, net, optimizer, criterion)
        if ep in lr_schedule:
            lr *= 0.1
            optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=0.0005)
        loss, acc = epoch('test', valloader, net, optimizer, criterion)
        print("Epoch {}: {}".format(ep, acc))

Epoch 0: 0.0818
Epoch 1: 0.08934285714285714
Epoch 2: 0.1013
Epoch 3: 0.10708571428571428
Epoch 4: 0.09412857142857142
Epoch 5: 0.08181428571428571
Epoch 6: 0.07404285714285715
Epoch 7: 0.07157142857142858
Epoch 8: 0.07142857142857142
Epoch 9: 0.07142857142857142
Epoch 10: 0.07142857142857142
Epoch 11: 0.07142857142857142
Epoch 12: 0.07142857142857142
Epoch 13: 0.07142857142857142
Epoch 14: 0.07142857142857142
Epoch 15: 0.07142857142857142
Epoch 16: 0.07142857142857142
Epoch 17: 0.07142857142857142
Epoch 18: 0.07142857142857142
Epoch 19: 0.07142857142857142
Epoch 20: 0.07142857142857142
Epoch 21: 0.07142857142857142
Epoch 22: 0.07142857142857142
Epoch 23: 0.07142857142857142
Epoch 24: 0.07142857142857142
Epoch 25: 0.07142857142857142
Epoch 26: 0.07142857142857142
Epoch 27: 0.07142857142857142
Epoch 28: 0.07142857142857142
Epoch 29: 0.07142857142857142
Epoch 30: 0.07142857142857142
Epoch 31: 0.07142857142857142
Epoch 32: 0.07142857142857142
Epoch 33: 0.07142857142857142
Epoch 34: 0.0714

In [23]:
epoch('test', valloader, net, optimizer, criterion)

(0.18464524394497275, 0.9452285714285714)